# External Data Sydony

## Sydony

https://www.nies.go.jp/whatsnew/20210506/20210506.html#:~:text=%E6%A3%AE%E6%9E%97%E7%81%AB%E7%81%BD%E3%81%8C%E6%B7%B1%E5%88%BB%E5%8C%96,%E7%81%AB%E7%81%BD%E3%81%AF%E5%8F%8E%E6%9D%9F%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82

EO: 
- https://apps.sentinel-hub.com/eo-browser/?zoom=13&lat=-33.8468&lng=150.35179&themeId=DEFAULT-THEME


In [2]:
import os
import warnings
import random
from pprint import pprint
import copy
from typing import List, Tuple
import glob
import json
import csv
# import dataclasses
from joblib import Parallel, delayed
from typing import List, Set, Dict, Any

from tqdm import tqdm
import numpy as np
import pandas as pd
from box import Box, from_file
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.optimize import minimize
from sklearn.metrics import jaccard_score
import cv2
import seaborn as sns

In [3]:
class CFG(object):
    # basic
    debug: bool = False
    folds: int  = 4
    seed: int   = 417
    eps: float  = 1e-12
    
    # data
    PATH_CSV: str  =  f'../data/035_Sydony/019_seed{seed}.csv'
    PATH_NPY_ROOT = f'../data/035_Sydony/'
    PATH_DATA = '../data/Sydony/'
    os.makedirs(PATH_DATA, exist_ok=True), os.makedirs(PATH_NPY_ROOT + 'img/', exist_ok=True)
    
# box
cfg = Box({k:v for k, v in dict(vars(CFG)).items() if '__' not in k})
    
# 乱数のシードを設定
np.random.seed(cfg.seed)
random.seed(cfg.seed)
    
pprint(cfg)

{'PATH_CSV': '../data/035_Sydony/019_seed417.csv',
 'PATH_DATA': '../data/Sydony/',
 'PATH_NPY_ROOT': '../data/035_Sydony/',
 'debug': False,
 'eps': 1e-12,
 'folds': 4,
 'seed': 417}


## CLI

In [3]:
!aws --version

aws-cli/1.27.131 Python/3.8.16 Linux/5.15.0-73-generic botocore/1.29.131


## Download
few cloud scene

In [4]:
PATHS_SENTILEL2_S3 = [
    # Sydony
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/3/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/5/29/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/5/24/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/5/19/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/5/14/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/5/9/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/5/4/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/4/19/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/4/9/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/30/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/15/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/",
    "s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/",  
]

In [8]:
for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    
    !aws s3 cp {PATH_S3} {OUTPUT_PATH} --recursive --request-payer
    # break

  0%|          | 0/15 [00:00<?, ?it/s]

56-H-KH-2023-6-23
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R10m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R10m/B03.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R10m/B02.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R20m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R10m/B04.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R10m/TCI.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R10m/B08.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/23/0/R20m/B01.jp2 to ../data/Sydony/56-H-KH-2023-6-23/R20m/B01.jp2
download: s3://sentinel-s2-l2a

  7%|▋         | 1/15 [03:05<43:11, 185.08s/it]

56-H-KH-2023-6-18
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R10m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R10m/B04.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R10m/B02.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R10m/B03.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R20m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R10m/TCI.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R20m/B01.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/6/18/0/R10m/B08.jp2 to ../data/Sydony/56-H-KH-2023-6-18/R10m/B08.jp2
download: s3://sentinel-s2-l2a

 13%|█▎        | 2/15 [06:24<41:55, 193.52s/it]

...

 87%|████████▋ | 13/15 [43:12<06:45, 202.76s/it]

56-H-KH-2023-3-5
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R10m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R10m/B02.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R10m/B03.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R10m/B04.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R20m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R10m/B08.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R20m/B01.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/5/0/R10m/TCI.jp2 to ../data/Sydony/56-H-KH-2023-3-5/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/20

 93%|█████████▎| 14/15 [46:34<03:22, 202.44s/it]

56-H-KH-2023-3-10
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R10m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R10m/B02.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R10m/B03.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R20m/AOT.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R10m/B04.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R10m/B08.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R10m/TCI.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/56/H/KH/2023/3/10/0/R20m/B01.jp2 to ../data/Sydony/56-H-KH-2023-3-10/R20m/B01.jp2
download: s3://sentinel-s2-l2a

100%|██████████| 15/15 [49:58<00:00, 199.89s/it]


## Loading data

In [5]:
BANDS = [
    'R20m/B01', 'R10m/B02', 'R10m/B03', 'R10m/B04', 'R20m/B05', 'R20m/B06', 
    'R20m/B07', 'R10m/B08', 'R20m/B8A', 'R60m/B09', 'R20m/B11', 'R20m/B12',
]

In [6]:
PATHS_NPY = []

for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    
    imgs = []
    
    for BAND in BANDS:
        PATH_BAND = f'{OUTPUT_PATH}{BAND}.jp2'
        img = cv2.imread(PATH_BAND, cv2.IMREAD_UNCHANGED)        
        if 'R20m' in BAND:
            img = cv2.resize(img, (img.shape[1]*2, img.shape[0]*2), interpolation=1)
        elif 'R60m' in BAND:
            img = cv2.resize(img, (img.shape[1]*6, img.shape[0]*6), interpolation=1)

        imgs.append(img)
        
    imgs = np.stack(imgs, axis=2)
    
    for h_idx in range(imgs.shape[0]//512):
        for w_idx in range(imgs.shape[1]//512):
            
            patch = imgs[h_idx*512:h_idx*512+512, w_idx*512:w_idx*512+512, :]
                
            PATH_NPY = f'{cfg.PATH_NPY_ROOT}img/{SCENE_ID}_h{h_idx}_w{w_idx}.npy'
            np.save(PATH_NPY, patch)
            PATHS_NPY.append(PATH_NPY)


  0%|          | 0/15 [00:00<?, ?it/s]

56-H-KH-2023-6-23


  7%|▋         | 1/15 [02:00<28:09, 120.65s/it]

56-H-KH-2023-6-18


 13%|█▎        | 2/15 [04:07<26:55, 124.25s/it]

56-H-KH-2023-6-3


In [ ]:
df = pd.DataFrame({
    'feature': PATHS_NPY
})

df['name'] = df['feature'].apply(lambda x: x.split('/')[-1].split('.')[0])
df['mask'] = False
df['defects'] = False
df['fold'] = -1
df['hight'] = 512
df['width'] = 512
df['california'] = False
df['cv'] = -1
df['area'] = 'Sydony'

df.head(4)

,feature,name,mask,defects,fold,hight,width,california,cv,area
0,../data/035_Sydony/img/56-H-KH-2023-6-23_h0_w0...,56-H-KH-2023-6-23_h0_w0,False,False,-1,512,512,False,-1,Sydony
1,../data/035_Sydony/img/56-H-KH-2023-6-23_h0_w1...,56-H-KH-2023-6-23_h0_w1,False,False,-1,512,512,False,-1,Sydony
2,../data/035_Sydony/img/56-H-KH-2023-6-23_h0_w2...,56-H-KH-2023-6-23_h0_w2,False,False,-1,512,512,False,-1,Sydony
3,../data/035_Sydony/img/56-H-KH-2023-6-23_h0_w3...,56-H-KH-2023-6-23_h0_w3,False,False,-1,512,512,False,-1,Sydony


In [ ]:
df.to_csv(cfg.PATH_CSV, index=False, header=True)